In [1]:
-- 17-1. Selling the Benefits

Commands completed successfully.

Total execution time: 00:00:00.003

Recite some of the benefits, which include the following:

<b>Stored procedures help centralize your Transact-SQL code in the data tier.</b> Web sites or applications that embed ad hoc SQL are notoriously difficult to modify in a production environment. When ad hoc SQL is embedded in an application, you may spend too much time trying to find and debug the embedded SQL. Once you’ve found the bug, chances are you’ll need to recompile the program executable, causing unnecessary application outages or application distribution nightmares. If you centralize your Transact-SQL code in stored procedures, you’ll have a centralized place to look for SQL code or SQL batches. If you document and standardize the code properly, your stored procedures will improve overall supportability of the application.

<b>Stored procedures can reduce network traffic for larger ad hoc queries.</b>Programming your application to execute a stored procedure, rather than push across a 500-line SQL call, can have a positive impact on your network and application performance, particularly if the call is repeated thousands of times a minute.

<b>Stored procedures encourage code reusability.</b> For example, if your web application uses a drop-down menu containing a list of cities and this drop-down is used in multiple web pages, you can call the stored procedure from each web page rather than embed the same SQL in multiple places.

<b>Stored procedures allow you to obscure the method of data retrieval.</b> If you change the underlying tables from which the source data is pulled, stored procedures (similar to views) can obscure this change from the application. This allows you to make changes without forcing a code change at the application tier. You can swap in new tables for the old, and as long as the same columns and data types are sent back to the application, the application is none the wiser.

<b>Stored procedures can do more than views</b>. They can take advantage of control-offlow techniques, temporary tables, table variables, and much more.

<b>Stored procedures have a stabilizing influence on query response time.</b> If you’ve worked extensively with ad hoc queries, you may have noticed that sometimes the amount of time it takes to return results from a query can vary wildly. This may be because of external factors, such as concurrent activity against the table (locking) or resource issues (memory, CPU). On the other hand, an ad hoc query may be performing erratically because SQL Server periodically chooses less efficient execution plans. With stored procedures, you gain more reliable query-plan caching and hence reuse. Notice that I use the word reliable here, rather than faster. Ad hoc queries can sometimes perform better than their stored procedure counterparts, but it all depends on the circumstances in which the execution plan was cached and how you have tested, tuned, and then implemented the code within.

In [2]:
-- 17-2. Creating a Stored Procedure

CREATE PROCEDURE dbo.ListCustomerNames
AS
    SELECT CustomerID,
        LastName,
        FirstName
    FROM Sales.Customer sc
        INNER JOIN Person.Person pp
            ON sc.CustomerID = pp.BusinessEntityID
    ORDER BY LastName,
    FirstName;

Commands completed successfully.

Total execution time: 00:00:00.045

In [3]:
-- 17-3. Generalizing a Stored Procedure

CREATE PROCEDURE dbo.LookupByAccount
(@AccountNumber VARCHAR(10),
@UpperFlag CHAR(1))
AS
    SELECT CASE UPPER(@UpperFlag)
            WHEN 'U' THEN UPPER(FirstName)
            ELSE FirstName
        END AS FirstName,
        CASE UPPER(@UpperFlag)
        WHEN 'U' THEN UPPER(LastName)
        ELSE LastName
        END AS LastName
FROM Person.Person
WHERE BusinessEntityID IN (SELECT CustomerID
    FROM Sales.Customer
    WHERE AccountNumber = @AccountNumber) ;

Commands completed successfully.

Total execution time: 00:00:00.011

In [4]:
-- 17-4. Making Parameters Optional

CREATE PROCEDURE dbo.LookupByAccount
(@AccountNumber VARCHAR(10),
@UpperFlag CHAR(1) = 'x')
AS
    SELECT CASE UPPER(@UpperFlag)
            WHEN 'U' THEN UPPER(FirstName)
            ELSE FirstName
        END AS FirstName,
        CASE UPPER(@UpperFlag)
            WHEN 'U' THEN UPPER(LastName)
            ELSE LastName
        END AS LastName
    FROM Person.Person
    WHERE BusinessEntityID IN (SELECT CustomerID
        FROM Sales.Customer
        WHERE AccountNumber = @AccountNumber);

: Msg 2714, Level 16, State 3, Procedure LookupByAccount, Line 3
There is already an object named 'LookupByAccount' in the database.

Total execution time: 00:00:00.044

In [5]:
-- 17-5. Making Early Parameters Optional

CREATE PROCEDURE dbo.LookupByAccount2
(@UpperFlag CHAR(1) = 'x',
@AccountNumber VARCHAR(10))
AS
    SELECT CASE UPPER(@UpperFlag)
            WHEN 'U' THEN UPPER(FirstName)
            ELSE FirstName
        END AS FirstName,
        CASE UPPER(@UpperFlag)
            WHEN 'U' THEN UPPER(LastName)
            ELSE LastName
        END AS LastName
    FROM Person.Person
    WHERE BusinessEntityID IN (SELECT CustomerID
        FROM Sales.Customer
        WHERE AccountNumber = @AccountNumber);

Commands completed successfully.

Total execution time: 00:00:00.012

In [6]:
-- 17-6. Returning Output

CREATE PROCEDURE dbo.EL_Department
    @GroupName NVARCHAR(50),
    @DeptCount INT OUTPUT
AS
    SELECT Name
    FROM HumanResources.Department
    WHERE GroupName = @GroupName
    ORDER BY Name;
    SELECT @DeptCount = @@ROWCOUNT;

Commands completed successfully.

Total execution time: 00:00:00.009

In [7]:
-- 17-7. Modifying a Stored Procedure

ALTER PROCEDURE dbo.SEL_Department
    @GroupName NVARCHAR(50)
AS
    SELECT Name
    FROM HumanResources.Department
    WHERE GroupName = @GroupName
    ORDER BY Name;
    SELECT @@ROWCOUNT AS DepartmentCount;

: Msg 208, Level 16, State 6, Procedure SEL_Department, Line 3
Invalid object name 'dbo.SEL_Department'.

Total execution time: 00:00:00.004

In [8]:
-- 17-8. Removing a Stored Procedure

DROP PROCEDURE dbo.SEL_Department;

: Msg 3701, Level 11, State 5, Line 3
Cannot drop the procedure 'dbo.SEL_Department', because it does not exist or you do not have permission.

Total execution time: 00:00:00.006

In [9]:
-- 17-9. Automatically Run a Stored Procedure at Start-Up

USE master;

CREATE TABLE dbo.SQLStartupLog
    (
    SQLStartupLogID INT IDENTITY(1, 1)
    NOT NULL
    PRIMARY KEY,
    StartupDateTime DATETIME NOT NULL
    );

Commands completed successfully.

Total execution time: 00:00:00.024

In [10]:
CREATE PROCEDURE dbo.INS_TrackSQLStartups
AS
    INSERT dbo.SQLStartupLog
        (StartupDateTime)
    VALUES (GETDATE()); 

Commands completed successfully.

Total execution time: 00:00:00.015

In [11]:
EXEC sp_procoption @ProcName = 'INS_TrackSQLStartups',
    @OptionName = 'startup', @OptionValue = 'true';

Commands completed successfully.

Total execution time: 00:00:03.377

In [12]:
-- 17-10. Viewing a Stored Procedure’s Definition

EXEC sp_helptext 'LookupByAccount';

Commands completed successfully.

Total execution time: 00:00:00.331

Text
-- 17-3. Generalizing a Stored Procedure
""
CREATE PROCEDURE dbo.LookupByAccount
"(@AccountNumber VARCHAR(10),"
@UpperFlag CHAR(1))
AS
SELECT CASE UPPER(@UpperFlag)
WHEN 'U' THEN UPPER(FirstName)
ELSE FirstName
"END AS FirstName,"


In [13]:
SELECT definition
FROM sys.sql_modules m
    INNER JOIN sys.objects o
    ON m.object_id = o.object_id
WHERE o.type = 'P'
    AND o.name = 'LookupByAccount';

(1 row affected)

Total execution time: 00:00:00.237

definition
"-- 17-3. Generalizing a Stored Procedure CREATE PROCEDURE dbo.LookupByAccount (@AccountNumber VARCHAR(10), @UpperFlag CHAR(1)) AS SELECT CASE UPPER(@UpperFlag) WHEN 'U' THEN UPPER(FirstName) ELSE FirstName END AS FirstName, CASE UPPER(@UpperFlag) WHEN 'U' THEN UPPER(LastName) ELSE LastName END AS LastName FROM Person.Person WHERE BusinessEntityID IN (SELECT CustomerID FROM Sales.Customer WHERE AccountNumber = @AccountNumber) ;"


In [14]:
-- 17-11. Documenting Stored Procedures

Commands completed successfully.

Total execution time: 00:00:00.003

<span style="font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre; color: rgb(0, 0, 255);">CREATE</span><span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;"></span><span style="font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre; color: rgb(0, 0, 255);">PROCEDURE</span><span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">dbo.IMP_DWP_FactOrder</span><span style="font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre; color: rgb(0, 0, 255);">AS</span>

<span style="color: #008000;">--&nbsp;Purpose:&nbsp;Populates&nbsp;the&nbsp;data&nbsp;warehouse,&nbsp;Called&nbsp;by&nbsp;Job</span>

<span style="color: #008000;">--&nbsp;Maintenance&nbsp;Log</span>

<span style="color: #008000;">--&nbsp;Update&nbsp;By&nbsp;Update&nbsp;Date</span>

<span style="color: #0000ff;">Description</span>

<span style="color: #008000;">--&nbsp;Joe&nbsp;Sack&nbsp;8/15/2008&nbsp;Created</span>

<span style="color: #008000;">--&nbsp;Joe&nbsp;Sack&nbsp;8/16/2008&nbsp;A&nbsp;new&nbsp;column&nbsp;was&nbsp;added&nbsp;to</span>

<span style="color: #008000;">--&nbsp;the&nbsp;base&nbsp;table,&nbsp;so&nbsp;it&nbsp;was&nbsp;added&nbsp;here&nbsp;as&nbsp;well.</span>

<span style="color: #008000;">--&nbsp;Transact-SQL&nbsp;code&nbsp;here</span>

In [15]:
-- 17-12. Determining the Current Nesting Level

-- First procedure
CREATE PROCEDURE dbo.QuickAndDirty
AS
SELECT @@NESTLEVEL;
GO

-- Second procedure
CREATE PROCEDURE dbo.Call_QuickAndDirty
AS
SELECT @@NESTLEVEL
EXEC dbo.QuickAndDirty;
GO

SELECT @@NESTLEVEL;
EXEC dbo.Call_QuickAndDirty;

Commands completed successfully.

Commands completed successfully.

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.042

(No column name)
0


(No column name)
1


(No column name)
2


In [16]:
-- 17-13. Encrypting a Stored Procedure

CREATE PROCEDURE dbo.SEL_EmployeePayHistory
    WITH ENCRYPTION
AS
    SELECT BusinessEntityID,
        RateChangeDate,
        Rate,
        PayFrequency,
        ModifiedDate
    FROM HumanResources.EmployeePayHistory;

Commands completed successfully.

Total execution time: 00:00:00.011

In [17]:
EXEC sp_helptext SEL_EmployeePayHistory;

The text for object 'SEL_EmployeePayHistory' is encrypted.

Total execution time: 00:00:00.006

In [18]:
SELECT definition
FROM sys.sql_modules m
    INNER JOIN sys.objects o
        ON m.object_id = o.object_id
WHERE o.type = 'P'
    AND o.name = 'SEL_EmployeePayHistory';

(1 row affected)

Total execution time: 00:00:00.024

definition
NULL


In [19]:
-- 17-14. Specifying a Security Context

CREATE PROCEDURE HumanResources.SEL_Department
    @GroupName NVARCHAR(50)
WITH EXECUTE AS OWNER
AS
    SELECT Name
    FROM HumanResources.Department
    WHERE GroupName = @GroupName
    ORDER BY Name;
    SELECT @@ROWCOUNT AS DepartmentCount;

: Msg 2760, Level 16, State 1, Procedure SEL_Department, Line 3
The specified schema name "HumanResources" either does not exist or you do not have permission to use it.

Total execution time: 00:00:00.003

In [20]:
-- 17-15. Avoiding Cached Query Plans

ALTER PROCEDURE dbo.LookupByAccount2
    (
    @UpperFlag VARCHAR(1) = 'x',
    @AccountNumber VARCHAR(10)
    )
    WITH RECOMPILE
AS
    SELECT CASE UPPER(@UpperFlag)
            WHEN 'U' THEN UPPER(FirstName)
            ELSE FirstName
        END AS FirstName,
        CASE UPPER(@UpperFlag)
            WHEN 'U' THEN UPPER(LastName)
            ELSE LastName
        END AS LastName
    FROM Person.Person
    WHERE BusinessEntityID IN (SELECT CustomerID
        FROM Sales.Customer
        WHERE AccountNumber = @AccountNumber);

Commands completed successfully.

Total execution time: 00:00:00.009

In [21]:
-- 17-16. Flushing the Procedure Cache

SELECT COUNT(*) 'CachedPlansBefore'
FROM sys.dm_exec_cached_plans;

(1 row affected)

Total execution time: 00:00:00.023

CachedPlansBefore
51


In [22]:
DBCC FREEPROCCACHE;
SELECT COUNT(*) 'CachedPlansAfter'
FROM sys.dm_exec_cached_plans;

DBCC execution completed. If DBCC printed error messages, contact your system administrator.

(1 row affected)

Total execution time: 00:00:00.087

CachedPlansAfter
1
